In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
train = pd.read_csv('C:/Users/Administrator/Desktop/移动广告反欺诈算法挑战/移动广告反欺诈算法挑战赛-0621/移动广告反欺诈算法挑战赛/round1_iflyad_anticheat_traindata.txt',encoding='utf-8',sep='\t')

In [3]:
test = pd.read_csv('C:/Users/Administrator/Desktop/移动广告反欺诈算法挑战/移动广告反欺诈算法挑战赛-0621/移动广告反欺诈算法挑战赛/round1_iflyad_anticheat_testdata_feature.txt',encoding='utf-8',sep='\t')

In [4]:
df = pd.concat([train,test])

H:\anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [5]:
#缺失值处理
#city用众数填充
df['city'] = df['city'].fillna('北京市')
#lan用众数填充
language = ['zh-CN','Zh-CN', 'zh-cn', 'cn', 'zh_CN', 'zh', 'ZH', 'CN', 'tw','zh_CN_#Hans','zh-TW','zh-HK','zh-US','zh_HK_#Hant',
           'zh-MO','zh-','zh_TW']
df.loc[df['lan'].isin(language),'lan'] = 'cn'
df['lan'] = df['lan'].fillna('cn')
#make用众数填充
df['make'] = df['make'].fillna('oppo')
#model用众数填充
df['model'] = df['model'].fillna('PBAMOO')
#osv用众数填充
df['osv'] = df['osv'].fillna('8.1.0')
#ver用众数填充
df['ver'] = df['ver'].fillna('30927000')

In [6]:
#不明白含义的特征，先删除
df.drop(['idfamd5','imeimd5','adidmd5','adunitshowid','openudidmd5','pkgname','macmd5','mediashowid','sid','h','w','reqrealip','ip',
        'ver','os'],axis=1,inplace=True)

In [7]:
#运营商变量替换
df.loc[df['carrier'] == 46000,'carrier'] = 1
df.loc[df['carrier'] == 46001,'carrier'] = 2
df.loc[df['carrier'] == 46003,'carrier'] = 3

In [8]:
#对nginxtime标准化
df['nginxtime_std'] = (df['nginxtime'] - df['nginxtime'].mean())/df['nginxtime'].std()
df = df.drop('nginxtime',axis=1)

In [9]:
df['make'] = df['make'].str.lower()

In [15]:
language = ['zh-CN','Zh-CN', 'zh-cn', 'cn', 'zh_CN', 'zh', 'ZH', 'CN', 'tw','zh_CN_#Hans','zh-TW','zh-HK','zh-US','zh_HK_#Hant',
           'zh-MO','zh-','zh_TW']
df.loc[df['lan'].isin(language),'lan'] = 'cn'

In [10]:
#将分类太多的特征取数量排前20的值，其他变成others，方便one-hot
citys = []
for i in df['city'].value_counts().head(20).index:
    citys.append(i)
df.loc[~df['city'].isin(citys),'city'] = 'others'

In [11]:
makes = []
for i in df['make'].value_counts().head(20).index:
    makes.append(i)
df.loc[~df['make'].isin(makes),'make'] = 'others'

In [12]:
models = []
for i in df['model'].value_counts().head(20).index:
    models.append(i)
df.loc[~df['model'].isin(models),'model'] = 'others'

In [13]:
osvs = []
for i in df['osv'].value_counts().head(20).index:
    osvs.append(i)
df.loc[~df['osv'].isin(osvs),'osv'] = 'others'

In [14]:
#one-hot处理
dm_city = pd.get_dummies(df['city'], prefix='city')
dm_lan = pd.get_dummies(df['lan'], prefix='lan')
dm_make = pd.get_dummies(df['make'], prefix='make')
dm_model = pd.get_dummies(df['model'], prefix='model')
dm_osv = pd.get_dummies(df['osv'], prefix='osv')

In [15]:
df = pd.concat([df,dm_city,dm_lan,dm_make,dm_model,dm_osv],axis=1)
df.drop(['city','lan','make','model','osv'],axis=1,inplace=True)

In [16]:
#拆回train和test
train_data = df[0:1000000]
test_data = df[1000000:].drop('label',axis=1)

In [21]:
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RFC
from xgboost import XGBClassifier

In [22]:
X = train_data.drop(['label'],axis=1)
y = train_data['label']

In [23]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8)

H:\anaconda\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [24]:
model3 = XGBClassifier()
model3.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [25]:
print(model3.score(X_test,y_test))

0.86192


In [76]:
model1 = LR()
model1.fit(X_train,y_train)

H:\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [115]:
model2 = RFC(n_estimators=50, max_depth=None,min_samples_split=20, random_state=0)
model2.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=20,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [116]:
print(model2.score(X_test,y_test))

0.901445


In [117]:
y_predict = model2.predict(test_data)

In [118]:
sid = test['sid']
result = pd.DataFrame({'sid':sid,'label':y_predict})
result.to_csv('result.csv')